## import the required libraries ##

In [105]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## request the data from url ##

In [106]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text

## using python beautifulsoup pull the data from XML ##

In [107]:
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')

## Give the column names ##

In [108]:
 
column_names = ['Postal Code','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

## append the data pulled into dataframe ##

In [109]:

for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

## Ignore cells with a borough that is Not assigned ##

In [110]:
df=df[df['Borough']!='Not assigned']

## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough ##

In [111]:
for i in range(0,df.shape[0]):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] = df.iloc[i][1]

In [112]:
#current version of the wiki page has neighborhoods merged(so we need not merged specially) by '/', so replacing it by ',' as suggested in the assignment
df['Neighborhood']=df['Neighborhood'].str.replace("/", ",")

In [113]:
df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern , Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill , Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [114]:
#data cleaning data(resetting the index values)

df = df.reset_index()
df = df.drop(['index'],axis=1)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [115]:
df.shape

(103, 3)

## Adding lat/lon columns to the dataframe ##

In [116]:
df_latlon = pd.read_csv("http://cocl.us/Geospatial_data")

In [117]:
df_latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [118]:
dff = pd.merge(df,df_latlon, on = 'Postal Code',how = 'outer')
dff.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
